# UCI MTB DH Data Retrieval

## Setup
#### Import Libraries

If you do not have these libraries available, you should install them using `pip`

```
pip install requests
pip install bs4
pip install pandas
```

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime as dt
import os

In [2]:
def calculate_age(born):
    today = dt.date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

Widen display area to prevent column wrapping, and always show all columns for debug 

In [3]:
pd.set_option('display.width', 2000)
pd.set_option('display.max_columns', None)

## Config

Which race data are we collecting?

1. Losinj
1. Fort William
1. Leogang
1. Val di Sole
1. Vallnord
1. Mont-Sainte-Anne
1. La Bresse

In [4]:
race = 1
gender = 'm'
event = 'dh'
quali = False

#### Data Sources

The UCI Live Timing API contains a lot of data points, but not all the ones we want (speed being the main one missing), and not even all the ones they include on their own PDF which is frustrating.

Similarly, Roots & Rain also has a lot of the data points, but again not all of them; most notably it's missing timing splits 4 and 5.

Therefore we need to pull from both sources and combine the sets.

We specify the URLs for both sources from which we will extract our data. The UCI API URL can be found by loading the Live Timing page then using your browser's inspector on the Network tab (in Chrome at least) to see the data feed. As the UCI seems to be using a Single Page Application (SPA) here, it's not straight forward to extract this link automagically.

**Note:** The Race list is now maintained as an external python config file `config.py` imported below

In [6]:
from config import races

racename = races[race]['name']
urlUci = races[race]['urls']['uci'] + str(( 3 if 'm' == gender else 6 ) - int(quali))
urlRoots = races[race]['urls']['rnr'] + gender

File handling setup

In [7]:
directory = event + str(race) + '_' + racename
if not os.path.exists(directory):
    os.makedirs(directory)

file_prefix = event + str(race) + '_' + racename + '_' + gender
file_prefix = os.path.join( directory, file_prefix )

# UCI API
### Load Data

These two lines make the actual request to the server, and then converts the JSON string response in to a usable list format (deserialization)

In [180]:
r = requests.get( urlUci )
d = r.json()

The API returns with three main sections:

1. `Last Finisher`
 - Racers in order of start time
2. `Results`
 - Racers in finishing rank order
3. `Riders`
 - Personal details on all racers
 
Each contains many data points. To see all the contained data, you can un-comment and execute any of the lines in the next section to explore more.

In [181]:
# display( d )
# display( d['Results'][7] )
# display( d['Riders']['1001'] )
# display( d['Results'][61] )

### Extract Data

Here we iterate over the `Results` sub-set of data to extract the information we care about: basically those that finished the race, some identifying info, and their splits.

If you looked at detail of the returned data set in the last step you might have noticed the rider's name is not stored next to their result, riders are only identified by a reference number. To facilitate our analysis later on it is useful to import each rider's name at this stage by cross-referencing the `Riders` sub-set.

First we find out the last man to drop-in's start number so we can use that to add a reverse order column.

In [182]:
lastStart = d['Riders'][list(d['Riders'].keys())[-1]]['StartOrder']

We start with an empty list `lst` and in each loop iteration add an entry (actually a dict) to that list for each rider.

In [183]:
splits = len(d['Results'][0]['Times'] )
lst = []
for idx, row in enumerate( d['Results'] ):
    fin = "Finished" == row['Status']
    res = {
        'rank': row['Position'] if fin else idx+1,
        'name': d['Riders'][str(row['RaceNr'])]['PrintName'],
        'id': row['RaceNr'],
        'uci': d['Riders'][str(row['RaceNr'])]['UciRiderId'],
        'bib': d['Riders'][str(row['RaceNr'])]['RaceNr'],
        'status': row['Status'],
        'speed': np.nan,
        'start': d['Riders'][str(row['RaceNr'])]['StartOrder'],
        'start_rev': lastStart - d['Riders'][str(row['RaceNr'])]['StartOrder'] +1
    }

    # Add all splits to result set
    for split in range( 0, splits ):
        res['split' + str(split+1)] = row['Times'][split]['RaceTime']/1000 if fin else np.nan

    # Append result set to list
    lst.append(res)

This line loads the completed list in to a Pandas dataframe so that we can easily write it out to CSV later on 

In [184]:
df = pd.DataFrame( lst )

#### Expand Dataset

Calculate and add all the extra columns we need for split and sector differences and their rankings

In [185]:
for i in range( 1, splits+1 ):
    split = 'split' + str(i)
    sector = split + '_sector'
    df[split + '_rank'] = df[split].rank(method='dense')
    df[split + '_vs_best'] = (df[split] - df[split].min())
    df[split + '_vs_winner'] = (df[split] - df[split][0])

    if i > 1:
        df[split + '_sector'] = df[split] - df['split' + str(i-1)]
        df[split + '_sector_rank'] = df[sector].rank(method='dense')
        df[split + '_sector_vs_best'] = (df[sector] - df[sector].min())
        df[split + '_sector_vs_winner'] = (df[sector] - df[sector][0])

We can take a peek at our data at this point to make sure it looks how we expect.

At this point the `speed` column is NaN (Not a Number) for all racers. This will be filled in below.

In [186]:
display( df['split1'][0], df['split1'].min() )

20.809999999999999

20.613

In [187]:
display( df.head(10) )

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner
0,1,1001,GWIN Aaron,1,NaN,20.810,52.860,80.785,116.514,139.193,63,2,Finished,10006516663,6.0,0.197,0.000,1.0,0.000,0.000,32.050,1.0,0.000,0.000,1.0,0.000,0.000,27.925,5.0,0.174,0.000,1.0,0.000,0.000,35.729,3.0,0.422,0.000,1.0,0.000,0.000,22.679,11.0,0.782,0.000
1,13,1013,SHAW Luca,2,NaN,20.613,54.162,82.787,118.094,139.991,60,5,Finished,10008813442,1.0,0.000,-0.197,3.0,1.302,1.302,33.549,6.0,1.499,1.499,6.0,2.002,2.002,28.625,11.0,0.874,0.700,3.0,1.580,1.580,35.307,1.0,0.000,-0.422,2.0,0.798,0.798,21.897,1.0,0.000,-0.782
2,16,1016,LUCAS Dean,3,NaN,20.680,53.964,81.870,117.830,140.328,64,1,Finished,10008103322,2.0,0.067,-0.130,2.0,1.104,1.104,33.284,3.0,1.234,1.234,2.0,1.085,1.085,27.906,4.0,0.155,-0.019,2.0,1.316,1.316,35.960,6.0,0.653,0.231,3.0,1.135,1.135,22.498,8.0,0.601,-0.181
3,19,1019,BLENKINSOP Samuel,4,NaN,21.224,54.699,82.891,118.749,141.107,61,4,Finished,10004485929,16.0,0.611,0.414,6.0,1.839,1.839,33.475,5.0,1.425,1.425,7.0,2.106,2.106,28.192,7.0,0.441,0.267,4.0,2.235,2.235,35.858,4.0,0.551,0.129,4.0,1.914,1.914,22.358,5.0,0.461,-0.321
4,34,1034,NORTON Dakotah,5,NaN,20.904,54.898,83.172,119.061,141.821,43,22,Finished,10010038167,9.0,0.291,0.094,9.0,2.038,2.038,33.994,12.0,1.944,1.944,9.0,2.387,2.387,28.274,8.0,0.523,0.349,5.0,2.547,2.547,35.889,5.0,0.582,0.160,5.0,2.628,2.628,22.760,14.0,0.863,0.081
5,7,1007,MOIR Jack,6,NaN,21.293,54.912,82.986,119.340,142.081,57,8,Finished,10008176575,18.0,0.680,0.483,10.0,2.052,2.052,33.619,7.0,1.569,1.569,8.0,2.201,2.201,28.074,6.0,0.323,0.149,6.0,2.826,2.826,36.354,10.0,1.047,0.625,6.0,2.888,2.888,22.741,13.0,0.844,0.062
6,9,1009,GREENLAND Laurie,7,NaN,21.152,54.779,83.420,119.614,142.191,58,7,Finished,10009404738,14.0,0.539,0.342,8.0,1.919,1.919,33.627,8.0,1.577,1.577,10.0,2.635,2.635,28.641,12.0,0.890,0.716,8.0,3.100,3.100,36.194,8.0,0.887,0.465,7.0,2.998,2.998,22.577,10.0,0.680,-0.102
7,5,1005,VERGIER Loris,8,NaN,21.076,56.543,84.294,119.956,142.271,49,16,Finished,10008723112,11.0,0.463,0.266,30.0,3.683,3.683,35.467,40.0,3.417,3.417,14.0,3.509,3.509,27.751,1.0,0.000,-0.174,10.0,3.442,3.442,35.662,2.0,0.355,-0.067,8.0,3.078,3.078,22.315,2.0,0.418,-0.364
8,2,1002,BROSNAN Troy,9,NaN,21.022,55.352,83.637,120.046,142.404,62,3,Finished,10007307417,10.0,0.409,0.212,14.0,2.492,2.492,34.330,18.0,2.280,2.280,11.0,2.852,2.852,28.285,9.0,0.534,0.360,11.0,3.532,3.532,36.409,12.0,1.102,0.680,9.0,3.211,3.211,22.358,4.0,0.461,-0.321
9,6,1006,HART Danny,10,NaN,20.704,54.741,82.511,119.447,142.965,54,11,Finished,10005470073,3.0,0.091,-0.106,7.0,1.881,1.881,34.037,13.0,1.987,1.987,4.0,1.726,1.726,27.770,2.0,0.019,-0.155,7.0,2.933,2.933,36.936,17.0,1.629,1.207,10.0,3.772,3.772,23.518,32.0,1.621,0.839


#### Rider Data

Saving the personal information about each racer is much easier as we can just export the entire `Riders` dataset. However, the rows and columns are the wrong way round so the `.T` command *transposes* the information, meaning it basically flips the axes.

In [188]:
df2 = pd.DataFrame( d['Riders'] )
df2 = df2.T
df2['Age'] = [ calculate_age( dt.datetime.strptime( dob[:10], "%Y-%m-%d" ) ) for dob in df2['BirthDate'] ]

Here we can glimpse the first few rows of our `DataFrame` and can check the data looks as we expect

In [189]:
display( df2.head() )

,BirthDate,CategoryCode,FamilyName,GivenName,Id,Nation,Outfit,PrintName,RaceId,RaceNr,ScoreboardName,StartOrder,StartTime,UciCode,UciRank,UciRiderId,UciTeamCode,UciTeamId,UciTeamName,WorldCupRank,Age
1001,1987-12-24T00:00:00,ME,GWIN,Aaron,101001,USA,NCh,GWIN Aaron,0,1,GWIN A,63,55620000,USA19871224,1,10006516663,YTM,1531,THE YT MOB,1,30
1002,1993-07-13T00:00:00,ME,BROSNAN,Troy,101002,AUS,NCh,BROSNAN Troy,0,2,BROSNAN T,62,55410000,AUS19930713,2,10007307417,CFT,2162,CANYON FACTORY DOWNHILL TEAM,2,24
1003,1981-11-13T00:00:00,ME,MINNAAR,Greg,101003,RSA,NaN,MINNAAR Greg,0,3,MINNAAR G,55,53940000,RSA19811113,5,10002818640,SCB,1307,SANTA CRUZ SYNDICATE,3,36
1004,1994-05-13T00:00:00,ME,BRUNI,Loic,101004,FRA,WCh,BRUNI Loic,0,4,BRUNI L,47,52320000,FRA19940513,3,10007544358,SGR,1667,SPECIALIZED GRAVITY,4,24
1005,1996-05-07T00:00:00,ME,VERGIER,Loris,101005,FRA,NaN,VERGIER Loris,0,5,VERGIER L,49,52680000,FRA19960507,7,10008723112,SCB,1307,SANTA CRUZ SYNDICATE,5,22


# Roots and Rain
### Load Data

Similar to the UCI api, we make a request to the server with the previously declared `urlRoots` variable. This time however we simply load the content of the response as text which is actually the HTML code of the web page. We do not do have a nice JSON API to read which means we will not deserialize.

Next we invoke a utility called `BeautifulSoup` to help us extract the data from this messy HTML code

In [190]:
r = requests.post( urlRoots )
c = r.content
soup = BeautifulSoup( c, "html.parser" )

### Extract Data

If you look at the Roots and Rain page you'll see it listed in a tabular format. What we do here is find all the rows of that table so we can extract the information we need.

Specifically we are looking for instances of `tr` (table row), with a class that *begins with* `c-` as this is a common denomenator I discovered when looking through the code with the browser inspector

In [191]:
rows = soup.find_all( "tr", class_=lambda x: x and 'c-' in x )

Similar to the UCI data set, here we will iterate over each row in our data set--basically each table row from the web page--and extract the bits we need.

Racer speed is the metric we're interested in, but in order to match that to our existing data set we need a corresponding identifier so we also extract the racer licence number as that exists in both sets and we can match them together: it is the *intersect* between both sets of data.

To summarise:
1. Extract licence number and corresponding speed
2. Import speed to existing DataFrame matching racers by licence

The `if` condition in the middle will exit this block of code once we hit the end of the Elite finishers, seeing as that's all we have in our existing data set so can't match anyone else

In [192]:
for row in rows:
    cells = row.find_all( "td" )

    speed = float(cells[12 if False == quali else 7].text[:5])
    licence = cells[4].text
    bib = int( cells[1].text )
    pos = cells[0].text[8:]
    if "" == pos: break

    # Match rider by UCI licence if present, otherwise fallback to bib
    if len(df2.loc[df2['UciRiderId'] == licence].index.values ):
        rid = int(df2.loc[df2['UciRiderId'] == licence].index.values[0])
    else:
        rid = int( df2.loc[df2['RaceNr'] == bib].index.values[0] )

    # Add speed, and other associated metrics
    df.loc[df['id'] == rid, 'speed'] = speed
    df.loc[df['id'] == rid, 'speed_ms'] = float(speed)*(1000/60/60)
    df.loc[df['id'] == rid, 'speed_ms_vs_best'] = df['speed_ms'].max() - df.speed_ms
    df['speed_rank'] = df.speed.rank(method='dense', ascending=False)

As before, we can take another look at how our data is looking, with the `speed` column now containing data 

In [193]:
display( df.head() )

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner,speed_ms,speed_ms_vs_best,speed_rank
0,1,1001,GWIN Aaron,1,46.93,20.810,52.860,80.785,116.514,139.193,63,2,Finished,10006516663,6.0,0.197,0.000,1.0,0.000,0.000,32.050,1.0,0.000,0.000,1.0,0.000,0.000,27.925,5.0,0.174,0.000,1.0,0.000,0.000,35.729,3.0,0.422,0.000,1.0,0.000,0.000,22.679,11.0,0.782,0.000,13.036111,0.000000,17.0
1,13,1013,SHAW Luca,2,48.92,20.613,54.162,82.787,118.094,139.991,60,5,Finished,10008813442,1.0,0.000,-0.197,3.0,1.302,1.302,33.549,6.0,1.499,1.499,6.0,2.002,2.002,28.625,11.0,0.874,0.700,3.0,1.580,1.580,35.307,1.0,0.000,-0.422,2.0,0.798,0.798,21.897,1.0,0.000,-0.782,13.588889,0.000000,1.0
2,16,1016,LUCAS Dean,3,47.95,20.680,53.964,81.870,117.830,140.328,64,1,Finished,10008103322,2.0,0.067,-0.130,2.0,1.104,1.104,33.284,3.0,1.234,1.234,2.0,1.085,1.085,27.906,4.0,0.155,-0.019,2.0,1.316,1.316,35.960,6.0,0.653,0.231,3.0,1.135,1.135,22.498,8.0,0.601,-0.181,13.319444,0.269444,6.0
3,19,1019,BLENKINSOP Samuel,4,48.00,21.224,54.699,82.891,118.749,141.107,61,4,Finished,10004485929,16.0,0.611,0.414,6.0,1.839,1.839,33.475,5.0,1.425,1.425,7.0,2.106,2.106,28.192,7.0,0.441,0.267,4.0,2.235,2.235,35.858,4.0,0.551,0.129,4.0,1.914,1.914,22.358,5.0,0.461,-0.321,13.333333,0.255556,4.0
4,34,1034,NORTON Dakotah,5,47.01,20.904,54.898,83.172,119.061,141.821,43,22,Finished,10010038167,9.0,0.291,0.094,9.0,2.038,2.038,33.994,12.0,1.944,1.944,9.0,2.387,2.387,28.274,8.0,0.523,0.349,5.0,2.547,2.547,35.889,5.0,0.582,0.160,5.0,2.628,2.628,22.760,14.0,0.863,0.081,13.058333,0.530556,16.0


# Points

Neither data set contains points awarded so we use a reference file and merge

Merge type here must be `outer` so people that finished outside the top 60, or DNF, don't get trimmed from the dataset

In [194]:
dfp = pd.read_csv( event + '_points_' + ( 'race_' if False == quali else 'qual_' ) + gender + '.csv', index_col=0 )
dfp = dfp.reset_index(drop=False)
df = df.merge( dfp, left_index=True, right_index=True, how="outer")

In [195]:
df.tail()

,bib,id,name,rank,speed,split1,split2,split3,split4,split5,start,start_rev,status,uci,split1_rank,split1_vs_best,split1_vs_winner,split2_rank,split2_vs_best,split2_vs_winner,split2_sector,split2_sector_rank,split2_sector_vs_best,split2_sector_vs_winner,split3_rank,split3_vs_best,split3_vs_winner,split3_sector,split3_sector_rank,split3_sector_vs_best,split3_sector_vs_winner,split4_rank,split4_vs_best,split4_vs_winner,split4_sector,split4_sector_rank,split4_sector_vs_best,split4_sector_vs_winner,split5_rank,split5_vs_best,split5_vs_winner,split5_sector,split5_sector_rank,split5_sector_vs_best,split5_sector_vs_winner,speed_ms,speed_ms_vs_best,speed_rank,points
59,116,1116,SUAREZ ALONSO Angel,60,34.17,21.978,57.445,94.660,155.377,186.814,35,30,Finished,10008831529,47.0,1.365,1.168,45.0,4.585,4.585,35.467,40.0,3.417,3.417,58.0,13.875,13.875,37.215,58.0,9.464,9.290,60.0,38.863,38.863,60.717,61.0,25.410,24.988,60.0,47.621,47.621,31.437,60.0,9.540,8.758,9.491667,4.097222,59.0,1.0
60,93,1093,NEWELL Jake,61,40.22,25.362,78.888,131.615,189.441,216.725,3,62,Finished,10007488582,62.0,4.749,4.552,61.0,26.028,26.028,53.526,59.0,21.476,21.476,61.0,50.830,50.830,52.727,61.0,24.976,24.802,61.0,72.927,72.927,57.826,60.0,22.519,22.097,61.0,77.532,77.532,27.284,57.0,5.387,4.605,11.172222,2.416667,56.0,NaN
61,151,1151,CIRIEGO Maxime,62,39.42,21.775,57.355,133.930,198.471,226.392,4,61,Finished,10009447073,38.0,1.162,0.965,42.0,4.495,4.495,35.580,43.0,3.530,3.530,62.0,53.145,53.145,76.575,62.0,48.824,48.650,62.0,81.957,81.957,64.541,62.0,29.234,28.812,62.0,87.199,87.199,27.921,58.0,6.024,5.242,10.950000,2.638889,57.0,NaN
62,24,1024,MACDONALD Brook,63,NaN,NaN,NaN,NaN,NaN,NaN,46,19,DNF,10006429969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,4,1004,BRUNI Loic,64,NaN,NaN,NaN,NaN,NaN,NaN,47,18,DNS,10007544358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Export

All that's left is to save our data to CSV files so we can quickly import it again for analysis and visualization without making constant requests to the online servers. This not only reduces load on the services providing the data, but also allows us to work on our analysis "offline", moreover giving us a local copy in case the results are ever taken down. It's also much quicker to load data this way than constantly hitting online servers.

In [196]:
df.id = df.id.astype(str)
dfm = df.merge( df2, left_on='id', right_index=True, how='inner' )

In [198]:
df.to_csv( file_prefix + '.results.csv' )
df2.to_csv( file_prefix + '.racers.csv' )
dfm.to_csv( file_prefix + '.merged.csv' )

--- 

## Credits

### Author: Dominic Wrapson


> **@domwrap**
<br>
<img src="https://png.icons8.com/material/24/000000/github-2.png">
<img src="https://png.icons8.com/material/24/000000/stackoverflow.png">
<img src="https://png.icons8.com/material/24/000000/linkedin.png">
<img src="https://png.icons8.com/material/24/000000/windows8.png">
<img src="https://png.icons8.com/ios-glyphs/24/000000/instagram-new.png">
<img src="https://png.icons8.com/material/24/000000/twitter.png">
<a href="https://medium.com/@domwrap"><img src="https://png.icons8.com/material/24/000000/medium-logo.png"></a>
>
> <img src="https://png.icons8.com/material/24/000000/home.png"> http://domwrap.me
>
><img src="https://png.icons8.com/material/24/000000/cycling-mountain-bike.png"> [Hwulex](https://www.pinkbike.com/u/Hwulex/)


---

#### Special Thanks

Mark Shilton for the inspiration
- http://lookatthestats.blogspot.ca
- https://plus.google.com/+MarkShilton
- https://dirtmountainbike.com/author/mrgeekstats


<a href="https://icons8.com">Icon pack by Icons8</a>